In [4]:

from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities  import ArxivAPIWrapper,WikipediaAPIWrapper


In [5]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki.name


'wikipedia'

In [8]:
arxiv_api_wrapper  = ArxivAPIWrapper(doc_content_chars_max=250,top_k_results=1)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name
tools=[wiki,arxiv]

In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
huggingfacehub_api_token = os.getenv("HUGGINGFACE_API_KEY")

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    
    

)
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
doc=loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200).split_documents(doc)
vectorstore = Chroma.from_documents(document,embeddings)
retriver = vectorstore.as_retriever()
retriver



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000025B512FDBA0>, search_kwargs={})

In [19]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(retriver,"langsmith_search","search any info about langsmith")
tools=[wiki,arxiv,retriver_tool]
retriver_tool.name
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\swimm\\OneDrive\\Desktop\\langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='search any info about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025B5644A0E0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000

In [20]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It" ,groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025B5EFD3190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025B5EFD0CD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [26]:
from langchain.agents import create_openai_tools_agent
agent  = create_openai_tools_agent(model,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [27]:
from langchain.agents import AgentExecutor

agent_exe = AgentExecutor(agent=agent,tools=tools,verbose=True)

agent_exe

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [28]:
res=agent_exe.invoke({"input":"tell me about langsmith"})




> Entering new AgentExecutor chain...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Invoking: `langsmith_search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​

For more see th

In [29]:
res

{'input': 'tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.  It has features for observability, evaluation, and prompt engineering. LangSmith can be used with or without LangChain or LangGraph. \n'}

In [30]:
res=agent_exe.invoke({"input":"who is the best wicketkeeper batsman of india"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'best wicketkeeper batsman of india'}`


Page: Wicket-keeper
Summary: In cricket, the wicket-keeper is the player on the fielding side who stands behind the wicket, ready to stop deliveries that pass the batsman, and take a catch, stump the batsman out, or run out a batsman when occasion ar
Invoking: `arxiv` with `{'query': 'best wicketkeeper batsman of india'}`
responded: That information doesn't answer who the best wicketkeeper batsman of India is.  Let's try a different approach.  



Published: 2021-02-03
Title: You Cannot Do That Ben Stokes: Dynamically Predicting Shot Type in Cricket Using a Personalized Deep Neural Network
Authors: Will Gürpınar-Morgan, Daniel Dinsdale, Joe Gallagher, Aditya Cherukumudi, Patrick Lucey
Summary:It seems like the tools provided aren't helpful for getting information about the best wicketkeeper batsman of India.  

I can give you some names that are often considered among